# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p> 
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>


In [1]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login
from dotenv import load_dotenv
import os

load_dotenv(override=True)
login(token=os.getenv("HF_TOKEN"))

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [2]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

# Generate with the base model

Here we will try out the base model which does not have a chat template.


In [3]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a


## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.


In [4]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
# TODO: 🦁 If your dataset is not in a format that TRL can convert to the chat template, you will need to process it. Refer to the [module](../chat_templates.md)
def process_dataset(sample):
    # TODO: 🐢 Convert the sample into a chat format
    # use the tokenizer's method to apply the chat template
    input_text = tokenizer.apply_chat_template(
        sample["messages"],
        add_generation_prompt=False,
        tokenize=False,
    )
    return {
        "formatted_chat": input_text,
    }


ds = ds.map(process_dataset)

In [6]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['full_topic', 'messages', 'formatted_chat'],
        num_rows: 2260
    })
    test: Dataset({
        features: ['full_topic', 'messages', 'formatted_chat'],
        num_rows: 119
    })
})


## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.


In [7]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

/Users/bytedance/workspace/opensource/smol-course/.venv/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/Users/bytedance/workspace/opensource/smol-course/.venv/lib/python3.11/site-packages/transformers/training_args.py:2248: UserWarning: `use_mps_device` is deprecated and will be removed in version 5.0 of 🤗 Transformers. `mps` device will be used by default if available similar to the way `cuda` device is used.Therefore, no action from user is required. 
  warnings.warn(
/Users/bytedance/workspace/opensource/smol-course/.venv/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/2260 [00:00<?, ? examples/s]

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.


In [8]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

  0%|          | 0/1000 [00:00<?, ?it/s]

{'loss': 1.6774, 'grad_norm': 2.720459461212158, 'learning_rate': 4.9500000000000004e-05, 'epoch': 0.02}
{'loss': 1.2563, 'grad_norm': 2.644317865371704, 'learning_rate': 4.9e-05, 'epoch': 0.04}
{'loss': 1.2339, 'grad_norm': 2.3945717811584473, 'learning_rate': 4.85e-05, 'epoch': 0.05}
{'loss': 1.161, 'grad_norm': 2.502696990966797, 'learning_rate': 4.8e-05, 'epoch': 0.07}
{'loss': 1.0657, 'grad_norm': 2.2998483180999756, 'learning_rate': 4.75e-05, 'epoch': 0.09}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.158974528312683, 'eval_runtime': 43.7803, 'eval_samples_per_second': 2.718, 'eval_steps_per_second': 0.343, 'epoch': 0.09}
{'loss': 1.1566, 'grad_norm': 2.2885255813598633, 'learning_rate': 4.7e-05, 'epoch': 0.11}
{'loss': 1.1648, 'grad_norm': 2.012310266494751, 'learning_rate': 4.6500000000000005e-05, 'epoch': 0.12}
{'loss': 1.1244, 'grad_norm': 2.128387451171875, 'learning_rate': 4.600000000000001e-05, 'epoch': 0.14}
{'loss': 1.086, 'grad_norm': 2.2242846488952637, 'learning_rate': 4.55e-05, 'epoch': 0.16}
{'loss': 1.1116, 'grad_norm': 2.080740213394165, 'learning_rate': 4.5e-05, 'epoch': 0.18}


  0%|          | 0/15 [00:00<?, ?it/s]

...

{'eval_loss': 1.0228322744369507, 'eval_runtime': 8.7641, 'eval_samples_per_second': 13.578, 'eval_steps_per_second': 1.712, 'epoch': 1.59}
{'loss': 0.8331, 'grad_norm': 1.707677960395813, 'learning_rate': 4.5e-06, 'epoch': 1.61}
{'loss': 0.8059, 'grad_norm': 1.804895281791687, 'learning_rate': 4.000000000000001e-06, 'epoch': 1.63}
{'loss': 0.8107, 'grad_norm': 2.071960687637329, 'learning_rate': 3.5000000000000004e-06, 'epoch': 1.65}
{'loss': 0.7932, 'grad_norm': 1.7174880504608154, 'learning_rate': 3e-06, 'epoch': 1.66}
{'loss': 0.8623, 'grad_norm': 1.9924761056900024, 'learning_rate': 2.5e-06, 'epoch': 1.68}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.0216660499572754, 'eval_runtime': 9.9001, 'eval_samples_per_second': 12.02, 'eval_steps_per_second': 1.515, 'epoch': 1.68}
{'loss': 0.8609, 'grad_norm': 1.8237817287445068, 'learning_rate': 2.0000000000000003e-06, 'epoch': 1.7}
{'loss': 0.7802, 'grad_norm': 1.7280333042144775, 'learning_rate': 1.5e-06, 'epoch': 1.72}
{'loss': 0.7957, 'grad_norm': 1.7798824310302734, 'learning_rate': 1.0000000000000002e-06, 'epoch': 1.73}
{'loss': 0.8345, 'grad_norm': 1.7968621253967285, 'learning_rate': 5.000000000000001e-07, 'epoch': 1.75}
{'loss': 0.7915, 'grad_norm': 1.7643896341323853, 'learning_rate': 0.0, 'epoch': 1.77}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.0214804410934448, 'eval_runtime': 9.3396, 'eval_samples_per_second': 12.741, 'eval_steps_per_second': 1.606, 'epoch': 1.77}
{'train_runtime': 2569.3532, 'train_samples_per_second': 1.557, 'train_steps_per_second': 0.389, 'train_loss': 0.9605698461532592, 'epoch': 1.77}


In [9]:
trainer.push_to_hub(tags=finetune_tags)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/odinushuaia/SmolLM2-FT-MyDataset/commit/6fdd51dc705f01717a0dd9996d0d690ac1b84121', commit_message='End of training', commit_description='', oid='6fdd51dc705f01717a0dd9996d0d690ac1b84121', pr_url=None, repo_url=RepoUrl('https://huggingface.co/odinushuaia/SmolLM2-FT-MyDataset', endpoint='https://huggingface.co', repo_type='model', repo_id='odinushuaia/SmolLM2-FT-MyDataset'), pr_revision=None, pr_num=None)

: 

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>


In [4]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]


# TODO: use the fine-tuned to model generate a response, just like with the base example.

# Load the model and tokenizer
ft_model_name = "odinushuaia/SmolLM2-FT-MyDataset"
ft_model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=ft_model_name
).to(device)
ft_tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=ft_model_name
)
# Generate response
formatted_prompt = ft_tokenizer.apply_chat_template(messages, tokenize=False)
inputs = ft_tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = ft_model.generate(**inputs, max_new_tokens=100)
print("after training:")
print(ft_tokenizer.decode(outputs[0], skip_special_tokens=True))

after training:
user
Write a haiku about programming
assistant
Hello! How can I help you today? I'm going to write a haiku about programming. What's programming? It's a type of art where you create a sequence of steps to solve a problem. Think of it like a recipe. You follow the steps, and then you see the result. Can you think of any programming languages? Python, Java, or JavaScript are all popular ones. What's the most popular programming language right now? Python is very popular,


## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.
